Importing packages

In [62]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

Getting raw data from wikipedia table into 'table_td' variable

In [63]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url)
soup = BeautifulSoup(source.content, 'lxml')
table = soup.find('table')
table_td = table.find_all('td')

Getting lists for each column. Cleaning data from unwanted characters like '(' or '\n'

In [64]:
PostalCode = []
Borough = []
Neighborhood = []
for row in table_td:
        PostalCode.append(row.text[1:4])
        Borough.append(row.text[4:].split('(')[0].replace('\n\n',''))
        if row.text[4:16] != 'Not assigned':
            if row.text.split('(')[1].replace(' /',',').replace('\n\n','')[-1]==')':
                Neighborhood.append(row.text.split('(')[1].replace(' /',',').replace('\n\n','').replace(')',''))
            else:
                Neighborhood.append(row.text.split('(')[1].replace(' /',',').replace('\n\n','').replace(')',' '))
        else:
            Neighborhood.append('')

Building a dataframe with the lists previously obtained

In [65]:
df = pd.DataFrame({"PostalCode" : PostalCode, "Borough" : Borough, "Neighborhood" : Neighborhood})
df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,
176,M6Z,Not assigned,
177,M7Z,Not assigned,
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Dropping rows with no borough assigned. Reset index

In [66]:
df = df.loc[df['Borough'] != 'Not assigned']
df.reset_index(drop=True, inplace=True)
df.shape

(103, 3)

In [67]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [68]:
df_coord = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
df_coord.shape

(103, 3)

In [69]:
df = df.sort_values('PostalCode')
df_coord = df_coord.sort_values('Postal Code')
df.reset_index(drop=True, inplace=True)
df_coord.reset_index(drop=True, inplace=True)
df_final = pd.concat([df,df_coord], axis=1)
df_final.drop(columns = ['Postal Code'], inplace=True)

In [70]:
df_final.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
